In [ ]:
import openai
import pandas as pd
import numpy as np
pd.options.display.max_colwidth=None

In [ ]:
openai.api_key = 'masked'

In [ ]:
testset=pd.read_csv('./data/test.tsv', sep = '\t')
testset.columns=['review', 'summary', 'kobart result', 'chatgpt-zero shot', 'chatgpt-few shot']
testset.head()

,review,summary,kobart result,chatgpt-zero shot,chatgpt-few shot
0,You don t need an additional holster when keeping the gun in this Works great Durable holds extra mags Nice and thin so you can pack one gun and it s stuff in each case,"Works great. Durable, holds extra mags",NaN,NaN,NaN
1,A great page turner that kept my interest until the last word I can t wait to read the next one,Great mixture of fantasy and modern times,NaN,NaN,NaN
2,after seeing the show I thought I would give the books a try Started off great but after book three it starts being more of a chore to read,after seeing the show I thought I would give the bookds a try.,NaN,NaN,NaN
3,It was nice to be able to test your tank and realize if the quality of water is acceptable The kit is complete and easy to use Buy with confidence,It was nice to be able to test your tank and realize if the quality of water is acceptable. The,NaN,NaN,NaN
4,This is where I get my B12 etc love the product also sprinkle once a day on dog s and kitten s food Oh yea most of use can use more vitamin B I use it on salads,"Love this product, has a cheesy flavor.",NaN,NaN,NaN


In [ ]:
#fewshot examples
import json
fewshot_examples=pd.read_csv('./data/fewshot_examples.csv').to_dict('records')
fewshot_examples=json.dumps(fewshot_examples)[1:-1]
fewshot_examples=fewshot_examples.replace('"review"', 'review').replace('"summary"', 'summary')
fewshot_examples

'{review: "I have cherished this book for many years and am pleased to have acquired the audiobook version. It was delivered promptly, in impeccable condition, and at an excellent cost.", summary: "Happy with audiobook purchase, fast and cheap"}, {review: "I replaced my Giant 29er MTB\'s weak plastic handlebar end caps with these. They fit well and should shield the handgrips nicely.", summary: "Replaced weak MTB handlebar caps with good fit."}, {review: "We have found this AA disposable battery to be the best for our Kodak camera. It is reasonably priced and comes highly recommended.", summary: "Top-rated, affordable AA disposable battery for Kodak camera."}, {review: "The product appears to be of good quality, but my dog is not a big fan of it. Although she eats it, she seems unsatisfied.", summary: "good quality, but my dog not satisfied with product"}, {review: "These hiking socks are excellent; they have kept my feet blister-free, warm, and dry. I plan to purchase more.", summary:

In [ ]:
#few-shot learning
from tqdm import trange
model = "gpt-3.5-turbo" 
for r in trange(len(testset)):
    
    review=testset.loc[r,"review"]
    messages = [ 
    {'role':'system', 'content': """Review refers to a customer's evaluation of a product or service posted on Amazon's e-commerce platform. Summary is a brief recap of the review. Character count should be less than 100."""}
    ] 
    question= """These are examples of creating a summary using a review """ + (fewshot_examples) + '.'
    question += """ please summarize the following review less than 100 characters. review:""" + review + ". summary:"
    messages.append({'role':'user', 'content':question})
    completion = openai.ChatCompletion.create(
            model=model,
            messages=messages, 
            temperature=0
    )
    
    summary=completion['choices'][0]['message']['content']
    testset.loc[r,'chatgpt-few shot']=summary

100%|███████████████████████████████████████| 1017/1017 [16:07<00:00,  1.05it/s]


In [ ]:
testset.head(10)

,review,summary,kobart result,chatgpt-zero shot,chatgpt-few shot
0,You don t need an additional holster when keeping the gun in this Works great Durable holds extra mags Nice and thin so you can pack one gun and it s stuff in each case,"Works great. Durable, holds extra mags",NaN,NaN,Durable gun case with extra mag storage.
1,A great page turner that kept my interest until the last word I can t wait to read the next one,Great mixture of fantasy and modern times,NaN,NaN,"Exciting page-turner, can't wait for the next one."
2,after seeing the show I thought I would give the books a try Started off great but after book three it starts being more of a chore to read,after seeing the show I thought I would give the bookds a try.,NaN,NaN,"Good start, but becomes a chore after book three."
3,It was nice to be able to test your tank and realize if the quality of water is acceptable The kit is complete and easy to use Buy with confidence,It was nice to be able to test your tank and realize if the quality of water is acceptable. The,NaN,NaN,"Complete and easy-to-use water testing kit, buy with confidence."
4,This is where I get my B12 etc love the product also sprinkle once a day on dog s and kitten s food Oh yea most of use can use more vitamin B I use it on salads,"Love this product, has a cheesy flavor.",NaN,NaN,"Versatile B12 supplement for humans and pets, great on salads."
5,Good solid feel to it and programmable Decisive yet quiet clicks of the buttons It is nothing but pure computing bliss,They don't make them like this anymore....,NaN,NaN,"Solid, programmable mouse with decisive, quiet clicks. Pure computing bliss."
6,If you don t require or don t like pocket clips on your pocket knives then this is the knife for you Very sharp out of box and holds its edge for a long time,Great EDC Knife for someone who doesn't require a pocket clip,NaN,NaN,Sharp knife without pocket clip.
7,Plenty meat and beans too Got another 8 pack in my cart Removable plastic cap is well vented and very easy to get on and off,"Very pleasing. Good consistency, hearty not skimpy. Plenty spicy.",NaN,NaN,Great canned food with easy-to-use cap.
8,It is very pretty and unique I like that it needs minimal care It will be easy to care for,Unique. I have not seen anything like it before.,NaN,NaN,"Pretty, unique plant with minimal care needed."
9,I ve loved this book for years and am glad to get hold of this MP3 audiobook This was shipped quickly in pristine condition and at a great price,"Perfect condition, shipped quickly",NaN,NaN,"Happy with fast, cheap audiobook purchase."


In [ ]:
fewshot=testset[['review','summary','chatgpt-few shot']]
fewshot=fewshot.dropna()
fewshot.head(10)

,review,summary,chatgpt-few shot
0,You don t need an additional holster when keeping the gun in this Works great Durable holds extra mags Nice and thin so you can pack one gun and it s stuff in each case,"Works great. Durable, holds extra mags",Durable gun case with extra mag storage.
1,A great page turner that kept my interest until the last word I can t wait to read the next one,Great mixture of fantasy and modern times,"Exciting page-turner, can't wait for the next one."
2,after seeing the show I thought I would give the books a try Started off great but after book three it starts being more of a chore to read,after seeing the show I thought I would give the bookds a try.,"Good start, but becomes a chore after book three."
3,It was nice to be able to test your tank and realize if the quality of water is acceptable The kit is complete and easy to use Buy with confidence,It was nice to be able to test your tank and realize if the quality of water is acceptable. The,"Complete and easy-to-use water testing kit, buy with confidence."
4,This is where I get my B12 etc love the product also sprinkle once a day on dog s and kitten s food Oh yea most of use can use more vitamin B I use it on salads,"Love this product, has a cheesy flavor.","Versatile B12 supplement for humans and pets, great on salads."
5,Good solid feel to it and programmable Decisive yet quiet clicks of the buttons It is nothing but pure computing bliss,They don't make them like this anymore....,"Solid, programmable mouse with decisive, quiet clicks. Pure computing bliss."
6,If you don t require or don t like pocket clips on your pocket knives then this is the knife for you Very sharp out of box and holds its edge for a long time,Great EDC Knife for someone who doesn't require a pocket clip,Sharp knife without pocket clip.
7,Plenty meat and beans too Got another 8 pack in my cart Removable plastic cap is well vented and very easy to get on and off,"Very pleasing. Good consistency, hearty not skimpy. Plenty spicy.",Great canned food with easy-to-use cap.
8,It is very pretty and unique I like that it needs minimal care It will be easy to care for,Unique. I have not seen anything like it before.,"Pretty, unique plant with minimal care needed."
9,I ve loved this book for years and am glad to get hold of this MP3 audiobook This was shipped quickly in pristine condition and at a great price,"Perfect condition, shipped quickly","Happy with fast, cheap audiobook purchase."
